In [5]:
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
#Importing DATA
conn = sqlite3.connect('data/raw_data_project_m1.db')

sql_statement='''
          SELECT personal_info.uuid,country_info.country_code, career_info.normalized_job_code, personal_info.age
          FROM personal_info
          JOIN career_info ON personal_info.uuid=career_info.uuid
          JOIN country_info ON personal_info.uuid=country_info.uuid
          JOIN poll_info ON personal_info.uuid=poll_info.uuid
          '''
raw_df=pd.DataFrame(pd.read_sql_query(sql_statement, conn))

raw_df.head()
# raw_df.info()

In [ ]:
#Cleaning age

#Exploring values
# age_values=raw_df['age'].unique()
# print(list(age_values))

# cleaning
clean_age=[]
for item in raw_df['age']:
    if "years old" in item:
        clean_age.append(int(item[0:2]))
    elif len(item)>=4:
        clean_age.append(2016-int(item))

raw_df['age']=clean_age #add to dataframe

raw_df['age']=raw_df['age']+4 #convert to 2020

In [ ]:
#cleaning age with lambda function

def clean_age(item):
    if "years old" in item:
        return int(item[0:2])
    elif len(item) == 4:
        return 2016 - int(item)

raw_df["age"] = raw_df.apply(lambda x: clean_age(x["age"]), axis=1)

    
raw_df.head()

In [ ]:
#Cleaning country_code

#Exploring values
# list(raw_df['country_code'].unique())
#Importing csv of country-country code
country_equiv=pd.read_csv('data/country_code.csv',names=['Country','Code'])

country_equiv['Country','Code'].to_dict()
#Converting to dictionary
country_dict={}
for i in country_equiv.index:
    k=country_equiv.loc[i,'Code']
    v=country_equiv.loc[i,'Country']
    country_dict[k[1:3]]=v
#Adding missing values
country_dict['GB']='Great Britain'
country_dict['GR']='Greece'
#Adding country to dataframe
for i in raw_df.index:
    code=raw_df.loc[i,'country_code']
    raw_df.loc[i,'country']=country_dict['code']

In [ ]:
url = 'http://api.dataatwork.org/v1/jobs/'
job_dict={}
search_val=list(raw_df['normalized_job_code'].unique())
import requests 

for val in search_val:
    if val==None:
        job_dict[val]=None
    else:
        data=val
        res = requests.get(url+data)
        job_title= res.json()
        job_dict[val]=job_title['title']
# job_dict
print('Finished!')

In [ ]:
job_dict2={}
with open('data/job_dict.csv', 'r') as t:
    reader=csv.reader(t)
    job_dict2={row[0]:row[1] for row in reader}

In [1]:
import csv
with open('data/job_dict.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(job_dict.items())


NameError: name 'job_dict' is not defined

In [ ]:
for i in raw_df.index:
    raw_df.loc[i,'Job Title']= job_dict[raw_df.loc[i,'normalized_job_code']]

In [ ]:
raw_df.info()
raw_df['Job Title'].

In [ ]:
job_counts=dict(raw_df['Job Title'].value_counts())
people_with_job=raw_df['Job Title'].count()


for i in raw_df.index:
    if pd.notnull(raw_df.loc[i,'Job Title']):
        raw_df.loc[i,'Quantity']= job_counts[raw_df.loc[i,'Job Title']]
        raw_df.loc[i,'Percentage']= job_counts[raw_df.loc[i,'Job Title']]*100/people_with_job

In [ ]:
raw_df.head()

PREPARING FOR BONUS

In [6]:
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm

In [7]:
conn = sqlite3.connect('data/raw_data_project_m1.db')

sql_statement='''
        SELECT personal_info.uuid,country_info.country_code, career_info.normalized_job_code, 
        personal_info.age, career_info.dem_education_level, 
        poll_info.question_bbi_2016wave4_basicincome_argumentsfor, 
        poll_info.question_bbi_2016wave4_basicincome_argumentsagainst, 
        poll_info.question_bbi_2016wave4_basicincome_vote
              
          FROM personal_info
          JOIN career_info ON personal_info.uuid=career_info.uuid
          JOIN country_info ON personal_info.uuid=country_info.uuid
          JOIN poll_info ON personal_info.uuid=poll_info.uuid
          '''
bonus_df=pd.DataFrame(pd.read_sql_query(sql_statement, conn))
bonus_df.head(10)

,uuid,country_code,normalized_job_code,age,dem_education_level,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,question_bbi_2016wave4_basicincome_vote
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,None,61 years old,no,None of the above,None of the above,I would not vote
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,861a9b9151e11362eb3c77ca914172d0,57 years old,high,It increases appreciation for household work a...,It might encourage people to stop working,I would probably vote for it
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,None,32 years old,None,It creates more equality of opportunity,Foreigners might come to my country and take a...,I would not vote
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,049a3f3a2b5f85cb2971ba77ad66e10c,45 years old,high,It reduces anxiety about financing basic needs,None of the above,I would probably vote for it
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,f4b2fb1aa40f661488e2782b6d57ad2f,41 years old,high,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,I would probably vote for it
5,28583060-dbe2-0133-ff9c-0a81e8b09a82,AT,27af8700f5577cec835acee2cb90a2ff,1990,high,It encourages financial independence and self-...,It is impossible to finance | It might encoura...,I would vote against it
6,b50dbb80-da53-0133-8956-0a81e8b09a82,AT,None,1990,medium,It reduces anxiety about financing basic needs,Foreigners might come to my country and take a...,I would probably vote for it
7,dd743bb0-d9a5-0133-53b2-0a81e8b09a82,AT,c1b670eba9ccb65e7c99f7da116d5b9c,47 years old,medium,It increases appreciation for household work a...,It might encourage people to stop working | Fo...,I would probably vote for it
8,4fde2370-da48-0133-3a42-0a81e8b09a82,AT,d71c4164c99a5f9bca773e755a72f40f,34 years old,medium,It creates more equality of opportunity | It r...,Foreigners might come to my country and take a...,I would probably vote for it
9,9949c4c0-da5f-0133-c832-0a81e8b09a82,AT,None,1994,no,"It increases solidarity, because it is funded ...",Foreigners might come to my country and take a...,I would vote for it


In [8]:
# Extracting job_title + job_skills
url = 'http://api.dataatwork.org/v1/jobs/'
job_dict={}
job_skills={}
search_val=bonus_df['normalized_job_code'].unique()
import requests 

for val in tqdm(search_val):
    if val==None:
        job_dict[val]=None
        job_skills[val]=None
    else:
        try:
            res = requests.get(url+val+'/related_skills')
            job_title=res.json()['job_title']
            job_dict[val] = job_title
            job_skills[val]={res.json()['skills'][i]['skill_name']:res.json()['skills'][i]['importance'] for i in range(len(res.json()['skills']))}
        except:
            res = requests.get(url+val)
            job_title=res.json()['title']
            job_dict[val] = job_title
            job_skills[val]=None
            
# job_skills 

100%|██████████| 157/157 [02:25<00:00,  1.08it/s]


In [14]:
trial=pd.DataFrame(job_skills).

bonus_df=bonus_df.merge(trial, how='left',left_on='normalized_job_code',right_index=True)

bonus_df

,uuid,country_code,normalized_job_code,age,dem_education_level,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,question_bbi_2016wave4_basicincome_vote,computers and electronics,english language,...,dynamic strength,stamina,extent flexibility,dynamic flexibility,gross body coordination,gross body equilibrium,night vision,peripheral vision,glare sensitivity,food production
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,None,61 years old,no,None of the above,None of the above,I would not vote,None,None,...,None,None,None,None,None,None,None,None,None,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,861a9b9151e11362eb3c77ca914172d0,57 years old,high,It increases appreciation for household work a...,It might encourage people to stop working,I would probably vote for it,4.61,4.01,...,1,1,1,1,1,1,1,1,1,1
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,None,32 years old,None,It creates more equality of opportunity,Foreigners might come to my country and take a...,I would not vote,None,None,...,None,None,None,None,None,None,None,None,None,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,049a3f3a2b5f85cb2971ba77ad66e10c,45 years old,high,It reduces anxiety about financing basic needs,None of the above,I would probably vote for it,3.45,3.55,...,1,1,1,1,1,1,1,1,1.12,1
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,f4b2fb1aa40f661488e2782b6d57ad2f,41 years old,high,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,I would probably vote for it,4.3,3.21,...,1,1,1.12,1,1,1,1,1,1,1.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,SK,847165cfda6b1dc82ae22b967da8af2f,37 years old,high,It reduces bureaucracy and administrative expe...,It is impossible to finance,I would probably vote for it,None,None,...,None,None,None,None,None,None,None,None,None,None
9645,39f989f0-db52-0133-8482-0a81e8b09a82,SK,a4d5b8b38f9513825d0d94a981ebe962,53 years old,high,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...,I would probably vote against it,4.3,3.21,...,1,1,1.12,1,1,1,1,1,1,1.06
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,SK,None,1992,low,It reduces anxiety about financing basic needs,None of the above,I would not vote,None,None,...,None,None,None,None,None,None,None,None,None,None
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,SK,775190277a849cba701b306a7b374c0a,47 years old,low,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...,I would vote for it,4.62,3.96,...,1.88,1,1,1,1,1,1,1,1,1


In [15]:
trial

,computers and electronics,english language,critical thinking,oral comprehension,problem sensitivity,information ordering,customer and personal service,mathematics,reading comprehension,active listening,...,dynamic strength,stamina,extent flexibility,dynamic flexibility,gross body coordination,gross body equilibrium,night vision,peripheral vision,glare sensitivity,food production
NaN,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
861a9b9151e11362eb3c77ca914172d0,4.61,4.01,3.75,3.62,3.62,3.62,3.53,3.52,3.5,3.5,...,1,1,1,1,1,1,1,1,1,1
049a3f3a2b5f85cb2971ba77ad66e10c,3.45,3.55,3.88,3.75,4,4.12,2.64,2.27,3.62,3.75,...,1,1,1,1,1,1,1,1,1.12,1
f4b2fb1aa40f661488e2782b6d57ad2f,4.3,3.21,3.5,3.5,3.75,3.62,3.71,3.19,3.38,3.38,...,1,1,1.12,1,1,1,1,1,1,1.06
27af8700f5577cec835acee2cb90a2ff,3.46,3.71,3.12,3.63,3.38,3.63,4.29,2.95,3.75,3.5,...,1.13,1,2.13,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b0fa6ede410f50b82ab74f5a705fe699,3.29,3.96,4,3.88,3.5,3.62,2.79,3.33,3.88,3.88,...,1,1,1,1,1,1,1.12,1,1,1.17
559a21f836c93876f31b60e6d10656a7,3.85,4.05,3.75,3.88,3.12,3.75,3.38,3.98,3.75,3.38,...,1.5,1,1,1,1,1,1,1,1,1.85
c1fb1a01b78373ac2153c66fa08d16dc,3.01,4.8,2.88,3.5,3.25,3,1.64,2.02,4.12,2.88,...,1,1,1,1,1,1,1,1,1,1
05bb9a333a66d6eb151e253623efe1c0,3.46,3.71,3.12,3.63,3.38,3.63,4.29,2.95,3.75,3.5,...,1.13,1,2.13,1,1,1,1,1,1,1


In [3]:
url = 'http://api.dataatwork.org/v1/jobs/'
job_dict={}
job_skills={}
search_val=list(bonus_df['normalized_job_code'].unique())
import requests 

data='19e384e977796c25780d8c28b12089c4' #error 404
data1='f4b2fb1aa40f661488e2782b6d57ad2f' #example
res = requests.get(url+data1+'/related_skills')
# res.json()

# job_title=res.json()['job_title']
# job_dict[val] = job_title
# job_skills[job_title]=[res.json()['skills'][i]['skill_name'] for i in range(len(res.json()['skills']))]

# job_skills

NameError: name 'bonus_df' is not defined

In [9]:
with open('data/job_skills.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(job_skills.items())

In [4]:
bonus_df['Job Skills'] = bonus_df.apply(lambda x: job_skills[x['normalized_job_code']],axis=1)

bonus_df.head()

,uuid,country_code,normalized_job_code,age,dem_education_level,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,question_bbi_2016wave4_basicincome_vote,Job Skills
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,None,61 years old,no,None of the above,None of the above,I would not vote,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,861a9b9151e11362eb3c77ca914172d0,57 years old,high,It increases appreciation for household work a...,It might encourage people to stop working,I would probably vote for it,"{'computers and electronics': 4.61, 'english l..."
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,None,32 years old,None,It creates more equality of opportunity,Foreigners might come to my country and take a...,I would not vote,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,049a3f3a2b5f85cb2971ba77ad66e10c,45 years old,high,It reduces anxiety about financing basic needs,None of the above,I would probably vote for it,"{'information ordering': 4.12, 'problem sensit..."
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,f4b2fb1aa40f661488e2782b6d57ad2f,41 years old,high,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,I would probably vote for it,"{'computers and electronics': 4.3, 'problem se..."


In [5]:
bonus_df.dropna(axis=0,subset=['dem_education_level','Job Skills'], inplace=True)

In [6]:
skill_list=[]
for list in tqdm(bonus_df['Job Skills']):
    if list!=None:
        for skill in list:
            if skill not in skill_list:
                skill_list.append(skill)


100%|██████████| 4883/4883 [00:00<00:00, 9478.99it/s]


In [7]:
bonus_df=pd.concat([bonus_df,pd.DataFrame(columns=skill_list, dtype=float)])

In [8]:
for i in tqdm(bonus_df.index):
    if bonus_df.loc[i,'Job Skills']!=None:
        for skill,importance in bonus_df.loc[i,'Job Skills'].items():
            bonus_df.loc[i,skill]=float(importance)
    


100%|██████████| 4883/4883 [02:11<00:00, 37.06it/s]


In [ ]:
bonus_df.tail()

In [10]:
bonus_df.columns
bonus_df.drop(['uuid', 'country_code', 'normalized_job_code', 'age','Job Skills',
       'question_bbi_2016wave4_basicincome_argumentsfor',
       'question_bbi_2016wave4_basicincome_argumentsagainst',
       'question_bbi_2016wave4_basicincome_vote'], axis=1, inplace=True)

In [9]:
bonus_df.describe()

,computers and electronics,english language,critical thinking,oral comprehension,problem sensitivity,information ordering,customer and personal service,mathematics,reading comprehension,active listening,...,dynamic strength,stamina,extent flexibility,dynamic flexibility,gross body coordination,gross body equilibrium,night vision,peripheral vision,glare sensitivity,food production
count,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,...,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000,4883.000000
mean,4.304311,3.631503,3.687602,3.736283,3.748177,3.652417,3.389812,3.225857,3.659934,3.580766,...,1.222984,1.065673,1.142412,1.002931,1.056363,1.052765,1.023522,1.021577,1.035259,1.064243
std,0.449819,0.339238,0.239209,0.214678,0.210110,0.171657,0.591357,0.432454,0.289866,0.226561,...,0.380973,0.242444,0.322825,0.024617,0.213988,0.215003,0.135783,0.144702,0.164321,0.171011
min,2.240000,2.320000,2.880000,3.000000,2.750000,3.000000,1.330000,1.980000,3.120000,2.880000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.300000,3.210000,3.500000,3.500000,3.620000,3.620000,3.010000,2.910000,3.380000,3.380000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,4.300000,3.680000,3.750000,3.750000,3.750000,3.620000,3.640000,3.190000,3.620000,3.500000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.040000
75%,4.620000,3.960000,3.880000,3.880000,3.880000,3.630000,3.710000,3.580000,3.880000,3.750000,...,1.500000,1.000000,1.120000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.060000
max,4.880000,4.800000,4.120000,4.120000,4.120000,4.120000,4.520000,4.910000,4.750000,4.120000,...,2.500000,2.880000,3.130000,1.250000,3.130000,3.000000,2.500000,2.250000,2.500000,3.210000


In [28]:
# bonus_df[bonus_df['dem_education_level']=='high'].info()

result=bonus_df.groupby(by='dem_education_level').mean()
top_skills_dict={}
for i in result.index:

    top_skills_dict[i]=result.T[i].sort_values(ascending=False)[:10]

    
pd.DataFrame(top_skills_dict).T

,computers and electronics,critical thinking,deductive reasoning,english language,information ordering,oral comprehension,oral expression,problem sensitivity,reading comprehension,written comprehension
high,4.298805,3.683588,3.668430,3.635597,3.652435,3.730037,3.633703,3.747359,3.652728,3.751446
low,4.297667,3.685768,3.667957,3.655290,3.658348,3.743565,3.641681,3.741391,3.668913,3.776623
medium,4.313975,3.693933,3.670968,3.618211,3.651464,3.740591,3.641757,3.752673,3.664430,3.758661
no,4.284896,3.666042,3.644687,3.632083,3.628333,3.740313,3.634896,3.726042,3.669896,3.763854


WEBSCRAPPING

In [ ]:
import requests

url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content


from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

table = soup.find('table')

import re
clean_string=re.sub(r'\<.+\>', '', str(table))
clean_string=re.sub(r'\n', '', clean_table)

countries=re.findall(r'[A-Z][a-z]+', clean_string)
# print(countries)

codes=re.findall(r'[A-Z]{2}', clean_string)
# print(codes)
import pandas as pd

# dict={
#     'countries':countries,
#     'codes': codes
# }


# country_codes=pd.DataFrame(dict)

# country_codes



In [22]:
'abc'.split(' ')

['abc']

In [26]:
bonus1['question_bbi_2016wave4_basicincome_argumentsfor']=bonus1.apply(lambda x: x['question_bbi_2016wave4_basicincome_argumentsfor'].split(' | '),axis=1)
bonus1



AttributeError: 'list' object has no attribute 'split'

In [31]:
arguments_pro=[]
for i in bonus1.index:
    for j in bonus1.loc[i,'question_bbi_2016wave4_basicincome_argumentsfor']:
        if j not in arguments_pro:
            arguments_pro.append(j)
            
arguments_pro.remove('None of the above')
arguments_pro


['It increases appreciation for household work and volunteering',
 'It encourages financial independence and self-responsibility',
 'It reduces anxiety about financing basic needs',
 'It creates more equality of opportunity',
 'It reduces bureaucracy and administrative expenses',
 'It increases solidarity, because it is funded by everyone']

In [35]:
def vote(x):
    if x=='I would probably vote for it' or x=='I would vote for it':
        return 'In Favor'
    
    elif x=='I would not vote':
        return 'Neutral'
    
    elif x=='I would probably vote against it' or x=='I would vote against it':
        return  'Against'
    

In [37]:
bonus1['Position']=bonus1.apply(lambda x: vote(x['question_bbi_2016wave4_basicincome_vote']),axis=1)

<ipython-input-37-edcb9e14df35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonus1['simplified_vote']=bonus1.apply(lambda x: vote(x['question_bbi_2016wave4_basicincome_vote']),axis=1)


In [41]:
def counter(arguments):
    i=0
    for argument in arguments:
        if argument in arguments_pro or argument in arguments_pro :
            i+=1
    return i

In [42]:
bonus1['Number of Pro arguments']=bonus1.apply(lambda x: counter(x['question_bbi_2016wave4_basicincome_argumentsfor']),axis=1)
bonus1['Number of Cons arguments']=bonus1.apply(lambda x: counter(x['question_bbi_2016wave4_basicincome_argumentsagainst']),axis=1)

<ipython-input-42-dd55fb40defe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonus1['Number of Pro arguments']=bonus1.apply(lambda x: counter(x['question_bbi_2016wave4_basicincome_argumentsfor']),axis=1)
<ipython-input-42-dd55fb40defe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonus1['Number of Cons arguments']=bonus1.apply(lambda x: counter(x['question_bbi_2016wave4_basicincome_argumentsagainst']),axis=1)


In [43]:
bonus1

,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,simplified_vote,Number of Pro arguments,Number of Cons arguments
0,I would not vote,[None of the above],None of the above,Neutral,0,0
1,I would probably vote for it,[It increases appreciation for household work ...,It might encourage people to stop working,In Favor,3,0
2,I would not vote,[It creates more equality of opportunity],Foreigners might come to my country and take a...,Neutral,1,0
3,I would probably vote for it,[It reduces anxiety about financing basic needs],None of the above,In Favor,1,0
4,I would probably vote for it,[It reduces anxiety about financing basic needs],It is impossible to finance | It might encoura...,In Favor,1,0
...,...,...,...,...,...,...
9644,I would probably vote for it,[It reduces bureaucracy and administrative exp...,It is impossible to finance,In Favor,1,0
9645,I would probably vote against it,[It reduces bureaucracy and administrative exp...,It might encourage people to stop working | On...,Against,1,0
9646,I would not vote,[It reduces anxiety about financing basic needs],None of the above,Neutral,1,0
9647,I would vote for it,[It reduces bureaucracy and administrative exp...,Foreigners might come to my country and take a...,In Favor,5,0


In [44]:
result=bonus1.groupby(by='simplified_vote').sum()
result

,Number of Pro arguments,Number of Cons arguments
simplified_vote,,
Against,2195,0
In Favor,11753,0
Neutral,623,0


In [2]:
import matplotlib.pyplot as plt

%matplotlib inline

TypeError: eval() arg 1 must be a string, bytes or code object